# Linear Optimization

Optimizes a linear straight-line path from a point on the viable brain surface region to a point on the subthalamic nucleus (STN) for DBS electrode planning.

Treats the following as obstacles:
- Sulci
- Ventricles
- Corpus Callosum (prevent brain crossing)

**Objective Functions**
- Distance to obstacles
- Minimum path distance  *(not optimized yet)*

## 1. Setup

### Import Libraries

In [ ]:
import os
import glob
import random
import numpy as np

import vedo
from vedo import Volume, show, Line, merge, Plotter

from scipy.optimize import minimize

### Configuration and Setup

In [ ]:
# Define new base directory and sub-folder structure
base_dir = "./FINAL_BRAIN_ATLAS/final_nii_files"

# Entry zones (viable surface regions for path start)
entry_zone_dir = os.path.join(base_dir, "entry_zone")
left_entry_zone_dir = os.path.join(entry_zone_dir, "LEFT_ENTRY_ZONE")
right_entry_zone_dir = os.path.join(entry_zone_dir, "RIGHT_ENTRY_ZONE")

# Subthalamic nucleus (targets)
stn_root_dir = os.path.join(base_dir, "subthalamic_nucleus")
left_stn_dir = os.path.join(stn_root_dir, "LEFT_STN")
right_stn_dir = os.path.join(stn_root_dir, "RIGHT_STN")

# Full brain (for context / outline only)
full_brain_dir = os.path.join(base_dir, "full_brain")

# Obstacles (same role as before)
obstacles_dir = os.path.join(base_dir, "obstacles")

print("Base directory:", base_dir)


### Load Brain Structures

Recursively load all `.nii` / `.nii.gz` files under each designated root:
- Entry Zones (LEFT/RIGHT) viable start regions (light brown, low alpha)
- STN Targets (LEFT/RIGHT) (distinct greens)
- Obstacles (red)
- Full Brain outline (very light grey, minimal alpha)

All subfolder contents are merged later per category.

In [ ]:
# Initialize lists for different structure types
left_entry_zone_structures = []
right_entry_zone_structures = []
left_stn_structures = []
right_stn_structures = []
allowed_left_entry_zone_structures = []
allowed_right_entry_zone_structures = []
obstacle_structures = []
full_brain_structures = []

# Loader utility: walk all subfolders and load every .nii / .nii.gz
def load_folder_surfaces(root_folder, color, alpha=1.0, outer_surf=False):
    loaded = []
    if not os.path.isdir(root_folder):
        print(f"[WARN] Folder not found: {root_folder}")
        return loaded
    
    if outer_surf:
        #do something
        for dirpath, dirnames, filenames in os.walk(root_folder):
            nii_files = [f for f in filenames if f.lower().endswith('.nii') or f.lower().endswith('.nii.gz')]
            for fname in nii_files:
                fpath = os.path.join(dirpath, fname)
                rel = os.path.relpath(fpath, root_folder)
                print(f"  Loading structure: {rel}")
                vol = Volume(fpath)
                surf = vol.isosurface()
                centre = surf.points.mean(axis=0)
                components = surf.split()
                outer_surfaces = []

                for comp in components:
                    comp.compute_normals(points=False, cells=True, consistency=True)
                    verts = np.array(comp.points)
                    faces = np.array(comp.cells)
                    norms = np.array(comp.celldata["Normals"])

                    faces_centre = verts[faces].mean(axis=1)
                    vectors = faces_centre - centre
                    dots = np.einsum("ij,ij->i", norms, vectors)
                    keep = dots > 0 

                    if keep.sum() == 0:
                        continue

                    outer_faces = faces[keep]
                    outer = vedo.Mesh([verts, outer_faces]) # Hadd to from vedo list
                    outer_surfaces.append(outer)
                outer_surfaces = merge(outer_surfaces)
                outer_surfaces.c(color).alpha(alpha)
                loaded.append(outer_surfaces)
    else:
        for dirpath, dirnames, filenames in os.walk(root_folder):
            nii_files = [f for f in filenames if f.lower().endswith('.nii') or f.lower().endswith('.nii.gz')]
            for fname in nii_files:
                fpath = os.path.join(dirpath, fname)
                rel = os.path.relpath(fpath, root_folder)
                print(f"  Loading structure: {rel}")
                vol = Volume(fpath)
                surf = vol.isosurface()
                surf.c(color).alpha(alpha)
                loaded.append(surf)
    if not loaded:
        print(f"  [INFO] No NIfTI files found under {root_folder}")
    return loaded

print("\nLoading Entry Zones (LEFT)...")
left_entry_zone_structures = load_folder_surfaces(left_entry_zone_dir, color=(0.95, 0.65, 0.85), alpha=0.04)
print("Loading Entry Zones (RIGHT)...")
right_entry_zone_structures = load_folder_surfaces(right_entry_zone_dir, color=(0.95, 0.75, 0.85), alpha=0.04)

print("\nLoading Entry Zones (LEFT)...")
allowed_left_entry_zone_structures = load_folder_surfaces(left_entry_zone_dir, color=(0.95, 0.65, 0.85), alpha=0.04, outer_surf=True)
print("Loading Entry Zones (RIGHT)...")
allowed_right_entry_zone_structures = load_folder_surfaces(right_entry_zone_dir, color=(0.95, 0.75, 0.85), alpha=0.04, outer_surf=True)

# STN folders (may already be deepest; recursion still safe)
print("\nLoading STN (LEFT)...")
left_stn_structures = load_folder_surfaces(left_stn_dir, color="green", alpha=1.0)
print("Loading STN (RIGHT)...")
right_stn_structures = load_folder_surfaces(right_stn_dir, color="green", alpha=1.0)

print("\nLoading Obstacles...")
obstacle_structures = load_folder_surfaces(obstacles_dir, color="red", alpha=0.7)

print("\nLoading Full Brain Outline...")
full_brain_structures = load_folder_surfaces(full_brain_dir, color="grey", alpha=0.01)

In [ ]:
# Plot and show the outer entry zone
actors1 = []
actors1 += allowed_left_entry_zone_structures
actors1 += allowed_right_entry_zone_structures
show(actors1, axes=1, viewup="z", title="Entry Zones w/ Normals")

In [ ]:
actors2 = []

# Add the meshes
actors2 += allowed_left_entry_zone_structures
actors2 += allowed_right_entry_zone_structures

# Add normals for all meshes
for mesh in allowed_left_entry_zone_structures + allowed_right_entry_zone_structures:
    centre = mesh.points.mean(axis=0)
    mesh.compute_normals(points=False, cells=True, consistency=True)
    verts = np.array(mesh.points)
    faces = np.array(mesh.cells)
    norms = np.array(mesh.celldata["Normals"])

    faces_centre = verts[faces].mean(axis=1)
    vectors = faces_centre - centre
    dots = np.einsum("ij,ij->i", norms, vectors)
    red = dots > 0 
    blue = ~red

    if red.any():
        arrows_red = vedo.Arrows(faces_centre[red], faces_centre[red] + norms[red]*5, c="red")
        actors2.append(arrows_red)
    if blue.any():
        arrows_blue = vedo.Arrows(faces_centre[blue], faces_centre[blue] + norms[blue]*5, c="blue")
        actors2.append(arrows_blue)
        print("Normal toward the centre is found")

# Show everything together
show(actors2, axes=1, viewup="z", title="Entry Zones with Normals")

### Merge Structures

Combine individual structures into single meshes for:
- LEFT/RIGHT Entry Zones (start sampling)
- LEFT/RIGHT STN (target sampling hemisphere-consistent)
- Obstacles (for intersection & distance)
- Full Brain (context outline only)


In [ ]:
# Merge structures into single meshes
merged_left_entry_zone = merge(left_entry_zone_structures) if left_entry_zone_structures else None
if merged_left_entry_zone: merged_left_entry_zone.alpha(0.04)

merged_right_entry_zone = merge(right_entry_zone_structures) if right_entry_zone_structures else None
if merged_right_entry_zone: merged_right_entry_zone.alpha(0.04)

merged_allowed_left_entry_zone = merge(allowed_left_entry_zone_structures) if allowed_left_entry_zone_structures else None
if merged_allowed_left_entry_zone: merged_allowed_left_entry_zone.alpha(0.04)

merged_allowed_right_entry_zone = merge(allowed_right_entry_zone_structures) if allowed_right_entry_zone_structures else None
if merged_allowed_right_entry_zone: merged_allowed_right_entry_zone.alpha(0.04)

merged_left_stn = merge(left_stn_structures) if left_stn_structures else None
merged_right_stn = merge(right_stn_structures) if right_stn_structures else None

merged_obstacles = merge(obstacle_structures) if obstacle_structures else None
if merged_obstacles: merged_obstacles.c("red").alpha(0.75)

merged_full_brain = merge(full_brain_structures) if full_brain_structures else None
if merged_full_brain: merged_full_brain.c("grey").alpha(0.01)

print("\nMerged Summary:")
print(f"  Left Entry Zone: {'yes' if merged_left_entry_zone else 'no'}")
print(f"  Right Entry Zone: {'yes' if merged_right_entry_zone else 'no'}")
print(f"  Left Allowed Entry Zone: {'yes' if merged_allowed_left_entry_zone else 'no'}")
print(f"  Right Allowed Entry Zone: {'yes' if merged_allowed_right_entry_zone else 'no'}")
print(f"  Left STN: {'yes' if merged_left_stn else 'no'}")
print(f"  Right STN: {'yes' if merged_right_stn else 'no'}")
print(f"  Obstacles: {'yes' if merged_obstacles else 'no'}")
print(f"  Full Brain: {'yes' if merged_full_brain else 'no'}")


### Helper Functions

In [ ]:
def compute_path_length(start_point, end_point):
    """Compute the distance between two points for the line path."""
    return np.linalg.norm(np.array(end_point) - np.array(start_point))


def compute_obstacle_distance(start_point, end_point):
    """Compute the minimum distance from the line path to any obstacle."""
    if merged_obstacles is None:
        return 0.0
    temp_line = Line([start_point, end_point])
    d = temp_line.distance_to(merged_obstacles)
    if isinstance(d, (list, tuple, np.ndarray)):
        return float(np.min(d))
    return float(d)


def check_intersection(start_point, end_point):
    """Check if the line path intersects any obstacle."""
    if merged_obstacles is None:
        return False
    intersections = merged_obstacles.intersect_with_line(start_point, end_point)
    return intersections is not None and len(intersections) > 0


def _closest_point_on_mesh(point, mesh):
    """Project a 3D point onto the closest point on a mesh surface."""
    if mesh is None:
        return tuple(point)
    try:
        cp = mesh.closest_point(point)
        return tuple(cp)
    except Exception:
        # Fallback to nearest vertex
        verts = np.asarray(mesh.vertices)
        if verts.size == 0:
            return tuple(point)
        idx = int(np.argmin(np.linalg.norm(verts - np.asarray(point), axis=1)))
        return tuple(verts[idx])


def project_to_surfaces(start_point, end_point, hemisphere):
    """Project start/end onto entry zone and STN surfaces for the chosen hemisphere."""
    if hemisphere == 'left':
        sp = _closest_point_on_mesh(start_point, merged_allowed_left_entry_zone)
        ep = _closest_point_on_mesh(end_point, merged_left_stn)
    else:
        sp = _closest_point_on_mesh(start_point, merged_allowed_right_entry_zone)
        ep = _closest_point_on_mesh(end_point, merged_right_stn)
    return sp, ep


def sample_start_and_end():
    """Sample a hemisphere-consistent start (entry zone) and end (STN)."""
    hemispheres = []
    if merged_allowed_left_entry_zone and merged_left_stn:
        hemispheres.append('left')
    if merged_allowed_right_entry_zone and merged_right_stn:
        hemispheres.append('right')
    if not hemispheres:
        raise RuntimeError("No valid hemisphere data available for sampling.")
    hemi = random.choice(hemispheres)
    if hemi == 'left':
        start = tuple(merged_allowed_left_entry_zone.generate_random_points(1).points[0])
        end = tuple(merged_left_stn.generate_random_points(1).points[0])
    else:
        start = tuple(merged_allowed_right_entry_zone.generate_random_points(1).points[0])
        end = tuple(merged_right_stn.generate_random_points(1).points[0])
    return hemi, start, end


## 2. Monte Carlo Optimization

### Path Generation and Optimization

Generate random paths from brain surface to STN and evaluate them:
- **Orange lines**: Failed paths (intersect obstacles)
- **Yellow lines**: Successful but suboptimal paths
- **Green line**: Optimal path (maximum distance from obstacles)

**Optimization Algorithm**

Runs until finding a successful path and after a minimum of `min_attempts`.

Chooses a **random location** from the viable brain surface region and the STN of the corresponding hemisphere. 

For each plot, the distance from obstacles is saved. The "best" path is determined using the highest distance to obstacles.

In [ ]:
# Configuration for Monte Carlo path generation
max_attempts = 1000
min_attempts = 100
successful_attempts = []  # Store (hemisphere, start_point, end_point, obstacle_distance, path_length) tuples
failed_lines = []

# Line width configuration (for visualization)
BEST_PATH_WIDTH = 8
SUBOPTIMAL_PATH_WIDTH = 5
FAILED_PATH_WIDTH = 5


In [ ]:
# Generate and evaluate hemisphere-consistent paths
for attempt in range(max_attempts):
    if attempt + 1 > min_attempts:
        break

    hemi, start_point, end_point = sample_start_and_end()
    print(f"Attempt {attempt+1}: hemisphere={hemi} start={start_point} end={end_point}")

    # Check for intersection if we have obstacles
    if merged_obstacles:
        if not check_intersection(start_point, end_point):
            # Compute metrics
            temp_line = Line([start_point, end_point])
            obstacle_dist = compute_obstacle_distance(start_point, end_point)
            path_len = compute_path_length(start_point, end_point)
            successful_attempts.append((hemi, start_point, end_point, obstacle_dist, path_len))
            print(f"  SUCCESS - Obstacle Distance: {obstacle_dist:.3f} Path Length: {path_len:.3f}")
        else:
            failed_line = Line([start_point, end_point]).c("orange").alpha(0.3).lw(FAILED_PATH_WIDTH)
            failed_lines.append(failed_line)
            print("  FAIL - Line intersects obstacles")
    else:
        obstacle_dist = 0.0
        path_len = compute_path_length(start_point, end_point)
        successful_attempts.append((hemi, start_point, end_point, obstacle_dist, path_len))
        print(f"  SUCCESS (no obstacles) Path Length: {path_len:.3f}")
else:
    print(f"Warning: Only found {len(successful_attempts)} non-intersecting lines after {max_attempts} attempts")


### Select Optimal Path

Find the best path among successful attempts by maximizing distance from obstacles.

In [ ]:
# Select best path (maximum obstacle distance; tie-breaker shorter path length)
best_path = None
suboptimal_lines = []
selected_hemisphere = None

if successful_attempts:
    # Sort: first by obstacle distance desc, then by path length asc
    successful_attempts.sort(key=lambda x: (-x[3], x[4]))

    best_hemi, best_start, best_end, best_obstacle_dist, best_path_len = successful_attempts[0]
    best_path = Line([best_start, best_end]).c("green").lw(BEST_PATH_WIDTH)
    selected_hemisphere = best_hemi
    print(f"\nBest path: hemisphere={best_hemi} obstacle_dist={best_obstacle_dist:.3f} path_len={best_path_len:.3f}")

    max_suboptimal_paths = 10
    s = 1
    for hemi, start_pt, end_pt, obstacle_dist, path_len in successful_attempts[1:]:
        if s > max_suboptimal_paths:
            break
        
        suboptimal_line = Line([start_pt, end_pt]).c("yellow").alpha(0.3).lw(SUBOPTIMAL_PATH_WIDTH)
        suboptimal_lines.append(suboptimal_line)
        print(f"Suboptimal path {s}: hemi={hemi} obstacle_dist={obstacle_dist:.3f} path_len={path_len:.3f}")
        
        s += 1
else:
    print("No successful paths found!")


## 3. Weighted Sum + Nelder-Mead Optimization

### Configuration

Set weights and optimization parameters:
- `w_obstacle`: Weight for obstacle distance (maximize)
- `w_path_length`: Weight for path length (minimize)
- Weights are normalized automatically

In [ ]:
# Weights for multi-objective optimization (easily configurable)
w_obstacle = 0.9     # Weight for obstacle distance (higher = prefer safer paths)
w_path_length = 0.1   # Weight for path length (higher = prefer shorter paths)

# Normalize weights to sum to 1
total_weight = w_obstacle + w_path_length
w_obstacle_norm = w_obstacle / total_weight
w_path_length_norm = w_path_length / total_weight

print(f"Objective function weights:\n  Obstacle distance: {w_obstacle_norm:.3f}\n  Path length: {w_path_length_norm:.3f}")

# Nelder-Mead parameters
initial_point_attempts = 50
nm_max_iterations = 5000
nm_tolerance = 1e-6  # default: 1e-6, applied as xatol and fatol

# Hemisphere lock for optimization (set later when picking x0)
nm_hemisphere = None

### Weighted Sum Objective Function

Combines both objectives (obstacle distance and path length) into a single scalar value for optimization.

In [ ]:
# Global variables for normalization (will be computed from sample paths)
max_obstacle_dist = None
max_path_length = None

def weighted_sum_objective(x):
    """
    Weighted sum objective function for optimization.

    Parameters:
    -----------
    x : array-like, shape (6,)
        [start_x, start_y, start_z, end_x, end_y, end_z]

    Returns:
    --------
    float : Objective value (to be minimized)
    """
    global nm_hemisphere
    start_guess = tuple(x[:3])
    end_guess = tuple(x[3:])

    # If hemisphere not set, infer a reasonable default
    if nm_hemisphere is None:
        nm_hemisphere = 'left' if merged_left_entry_zone and merged_left_stn else 'right'

    # Project guesses to valid surfaces for the locked hemisphere
    start_point, end_point = project_to_surfaces(start_guess, end_guess, nm_hemisphere)

    # Hard constraint: penalize intersection
    if check_intersection(start_point, end_point):
        return 1e10

    # Compute objectives
    obstacle_dist = compute_obstacle_distance(start_point, end_point)
    path_length = compute_path_length(start_point, end_point)

    # Normalize objectives
    if max_obstacle_dist is not None and max_obstacle_dist > 0:
        norm_obstacle_dist = obstacle_dist / max_obstacle_dist
    else:
        norm_obstacle_dist = obstacle_dist / 100.0

    if max_path_length is not None and max_path_length > 0:
        norm_path_length = path_length / max_path_length
    else:
        norm_path_length = path_length / 200.0

    # Weighted sum: minimize (-obstacle_dist) and minimize path_length
    objective = -w_obstacle_norm * norm_obstacle_dist + w_path_length_norm * norm_path_length
    return objective

### Estimate Normalization Constants

Sample a few random paths to estimate typical ranges for obstacle distance and path length.

In [ ]:
ESTIMATION_SAMPLES = 100

In [ ]:
# Sample random paths to estimate normalization constants
print("Estimating normalization constants from random samples...")
sample_obstacle_dists = []
sample_path_lengths = []

for _ in range(ESTIMATION_SAMPLES):
    print(f"Sampling path {_+1}/{ESTIMATION_SAMPLES}...")
    hemi, start, end = sample_start_and_end()
    if not check_intersection(start, end):
        obstacle_dist = compute_obstacle_distance(start, end)
        path_length = compute_path_length(start, end)
        sample_obstacle_dists.append(obstacle_dist)
        sample_path_lengths.append(path_length)
        # print(f"  obstacle_dist={obstacle_dist:.3f} path_len={path_length:.3f}")
    else:
        print("  Sampled path intersects obstacles, skipping...")

if sample_obstacle_dists:
    max_obstacle_dist = max(sample_obstacle_dists)
    max_path_length = max(sample_path_lengths)
    print(f"Estimated max obstacle distance: {max_obstacle_dist:.2f}")
    print(f"Estimated max path length: {max_path_length:.2f}")
else:
    print("Warning: No valid paths found for normalization, using defaults")
    max_obstacle_dist = 8
    max_path_length = 100


In [ ]:
## OPTION 2: Use Monte Carlo results for normalization (if available)
if successful_attempts:
    print("\nUsing Monte Carlo results for normalization...")
    mc_obstacle_dists = [x[3] for x in successful_attempts]
    mc_path_lengths = [x[4] for x in successful_attempts]
    max_obstacle_dist = max(mc_obstacle_dists)
    max_path_length = max(mc_path_lengths)
    print(f"Max obstacle distance from Monte Carlo: {max_obstacle_dist:.2f}")
    print(f"Max path length from Monte Carlo: {max_path_length:.2f}")
else:
    print("\nNo Monte Carlo results available for normalization, using defaults")
    max_obstacle_dist = 8
    max_path_length = 100


### Set Initial Path for Optimization 

In [ ]:
## OPTION 1: Find first feasible initial point from random sampling
print("\nFinding feasible initial point...")
x0 = None
for attempt in range(initial_point_attempts):
    hemi, start, end = sample_start_and_end()
    if not check_intersection(start, end):
        x0 = np.concatenate([start, end])
        nm_hemisphere = hemi  # lock hemisphere for optimization
        print(f"Found feasible starting point after {attempt+1} attempts (hemi={hemi})")
        print(f"  Start: {start}")
        print(f"  End: {end}")
        break

if x0 is None:
    print("Warning: Could not find feasible starting point, using last sampled anyway")
    hemi, start, end = sample_start_and_end()
    x0 = np.concatenate([start, end])
    nm_hemisphere = hemi

In [ ]:
## OPTION 2: Select best Monte Carlo path as starting point (if available)
if successful_attempts:
    best_hemi, best_start, best_end, best_obstacle_dist, best_path_len = successful_attempts[0]
    print("\nUsing best Monte Carlo path as initial point for optimization...")
    x0 = np.concatenate([best_start, best_end])
    nm_hemisphere = best_hemi
    selected_hemisphere = best_hemi
else:
    print("\nNo Monte Carlo successful paths available; retaining previous x0 and hemisphere.")

### Run Nelder-Mead Optimization

Use Nelder-Mead simplex algorithm to optimize the weighted sum objective. Starts from a random feasible point.

In [ ]:
# Run Nelder-Mead optimization
print(f"\nRunning Nelder-Mead optimization (hemi={nm_hemisphere})...")
print(f"  Max iterations: {nm_max_iterations}")
print(f"  Tolerance: {nm_tolerance}")

result = minimize(
    weighted_sum_objective,
    x0,
    method='Nelder-Mead',
    options={
        'maxiter': nm_max_iterations,
        'xatol': nm_tolerance,
        'fatol': nm_tolerance,
        'disp': True
    }
)

raw_start = tuple(result.x[:3])
raw_end = tuple(result.x[3:])
nm_optimal_start, nm_optimal_end = project_to_surfaces(raw_start, raw_end, nm_hemisphere)
nm_optimal_objective = result.fun
selected_hemisphere = nm_hemisphere

print(f"\n{'='*60}")
print(f"Optimization Results:")
print(f"{'='*60}")
print(f"Success: {result.success}")
print(f"Message: {result.message}")
print(f"Iterations: {result.nit}")
print(f"Function evaluations: {result.nfev}")
print(f"Final objective value: {nm_optimal_objective:.6f}")
print(f"\nOptimal path (projected, hemi={nm_hemisphere}):")
print(f"  Start: {nm_optimal_start}")
print(f"  End: {nm_optimal_end}")

### Evaluate Optimal Path

Compute detailed metrics for the optimized path.

In [ ]:
# Evaluate the optimal path
nm_obstacle_dist = compute_obstacle_distance(nm_optimal_start, nm_optimal_end)
nm_path_length = compute_path_length(nm_optimal_start, nm_optimal_end)
nm_has_intersection = check_intersection(nm_optimal_start, nm_optimal_end)

print(f"\nOptimal Path Metrics:")
print(f"  Obstacle distance: {nm_obstacle_dist:.3f}")
print(f"  Path length: {nm_path_length:.3f}")
print(f"  Intersects obstacles: {nm_has_intersection}")

# Create line for visualization
nm_optimal_line = Line([nm_optimal_start, nm_optimal_end]).c("cyan").lw(10)
print(f"\nOptimal path created (cyan line)")

### Compare with Monte Carlo Results

Compare the Nelder-Mead optimized path with the best Monte Carlo path.

In [ ]:
if successful_attempts and nm_optimal_line is not None:
    mc_best_hemi, mc_best_start, mc_best_end, mc_best_obstacle_dist, mc_best_path_len = successful_attempts[0]
    mc_best_path_length = compute_path_length(mc_best_start, mc_best_end)
    
    # Compute weighted sum for Monte Carlo best
    norm_mc_obstacle = mc_best_obstacle_dist / max_obstacle_dist if max_obstacle_dist > 0 else 0
    norm_mc_length = mc_best_path_length / max_path_length if max_path_length > 0 else 0
    mc_weighted_sum = -w_obstacle_norm * norm_mc_obstacle + w_path_length_norm * norm_mc_length
    
    # Compute weighted sum for Nelder-Mead
    norm_nm_obstacle = nm_obstacle_dist / max_obstacle_dist if max_obstacle_dist > 0 else 0
    norm_nm_length = nm_path_length / max_path_length if max_path_length > 0 else 0
    nm_weighted_sum = -w_obstacle_norm * norm_nm_obstacle + w_path_length_norm * norm_nm_length
    
    print(f"\n{'='*60}")
    print(f"Comparison: Monte Carlo vs Weighted Sum Optimization")
    print(f"{'='*60}")
    print(f"\nMonte Carlo (best of {len(successful_attempts)} successful):")
    print(f"  Obstacle distance: {mc_best_obstacle_dist:.3f}")
    print(f"  Path length: {mc_best_path_length:.3f}")
    print(f"  Weighted sum: {mc_weighted_sum:.6f}")
    
    print(f"\nWeighted Sum Optimization:")
    print(f"  Obstacle distance: {nm_obstacle_dist:.3f}")
    print(f"  Path length: {nm_path_length:.3f}")
    print(f"  Weighted sum: {nm_weighted_sum:.6f}")
    
    print(f"\nImprovement:")
    obstacle_improvement = ((nm_obstacle_dist - mc_best_obstacle_dist) / mc_best_obstacle_dist * 100) if mc_best_obstacle_dist > 0 else 0
    length_improvement = ((mc_best_path_length - nm_path_length) / mc_best_path_length * 100) if mc_best_path_length > 0 else 0
    weighted_improvement = ((mc_weighted_sum - nm_weighted_sum) / abs(mc_weighted_sum) * 100) if mc_weighted_sum != 0 else 0
    
    print(f"  Obstacle distance: {obstacle_improvement:+.2f}%")
    print(f"  Path length: {length_improvement:+.2f}% (shorter is better)")
    print(f"  Weighted sum objective: {weighted_improvement:+.2f}% (lower is better)")
else:
    print("No Monte Carlo results available for comparison")


## Visualization

In [ ]:
actors = []
if merged_full_brain: actors.append(merged_full_brain)

if selected_hemisphere == 'left':
    if merged_left_entry_zone:
        actors.append(merged_left_entry_zone.clone().alpha(0.08))
    if merged_right_entry_zone:
        actors.append(merged_right_entry_zone.clone().alpha(0.02))
    if merged_left_stn:
        actors.append(merged_left_stn.clone().alpha(0.9))
    if merged_right_stn:
        actors.append(merged_right_stn.clone().alpha(0.3))
elif selected_hemisphere == 'right':
    if merged_left_entry_zone:
        actors.append(merged_left_entry_zone.clone().alpha(0.02))
    if merged_right_entry_zone:
        actors.append(merged_right_entry_zone.clone().alpha(0.08))
    if merged_left_stn:
        actors.append(merged_left_stn.clone().alpha(0.3))
    if merged_right_stn:
        actors.append(merged_right_stn.clone().alpha(0.9))
else:
    if merged_left_entry_zone: actors.append(merged_left_entry_zone)
    if merged_right_entry_zone: actors.append(merged_right_entry_zone)
    if merged_left_stn: actors.append(merged_left_stn)
    if merged_right_stn: actors.append(merged_right_stn)

if merged_obstacles: actors.append(merged_obstacles)

actors.extend(failed_lines)
actors.extend(suboptimal_lines)
if best_path:
    mc_best_faded = best_path.clone().alpha(0.3)
    actors.append(mc_best_faded)

try:
    actors.append(nm_optimal_line)
except:
    print("Nelder-Mead optimized path not found.")

print(f"Rendering scene with optimized path (cyan)...")
show(actors, axes=1, viewup="z", 
     title=f"Optimized Path", new=True, interactive=True)


#### K3D

In [ ]:
import k3d

# Helper function to convert vedo mesh to k3d mesh
def vedo_to_k3d_mesh(vedo_mesh, color=None, opacity_factor=1.0):
    vertices = np.array(vedo_mesh.vertices, dtype=np.float32)
    faces = np.array(vedo_mesh.cells, dtype=np.uint32)
    
    if color is None:
        vedo_color = vedo_mesh.color()
        if isinstance(vedo_color, str):
            import matplotlib.colors as mcolors
            if vedo_color in mcolors.CSS4_COLORS:
                color_hex = mcolors.CSS4_COLORS[vedo_color]
                color = int(color_hex.lstrip('#'), 16)
            else:
                color = 0x808080
        elif isinstance(vedo_color, (list, tuple, np.ndarray)):
            # Handle RGB tuples/arrays (values 0-1 or 0-255)
            rgb = np.asarray(vedo_color[:3])
            # Normalize to 0-255 range if needed
            if rgb.max() <= 1.0:
                rgb = (rgb * 255).astype(int)
            else:
                rgb = rgb.astype(int)
            r, g, b = int(rgb[0]), int(rgb[1]), int(rgb[2])
            color = (r << 16) | (g << 8) | b
        else:
            color = 0x808080
    
    vedo_alpha = vedo_mesh.alpha()
    if vedo_alpha is not None:
        opacity = float(vedo_alpha)
    else:
        opacity = 1.0
        
    opacity = np.clip(opacity * opacity_factor, 0.0, 1.0)
    
    return k3d.mesh(vertices, faces, color=color, opacity=opacity)

def vedo_line_to_k3d(vedo_line, color=None, width=0.1):
    line_scale_factor = 0.2
    points = np.array(vedo_line.vertices, dtype=np.float32)
    
    if color is None:
        vedo_color = vedo_line.color()
        if isinstance(vedo_color, str):
            import matplotlib.colors as mcolors
            if vedo_color in mcolors.CSS4_COLORS:
                color_hex = mcolors.CSS4_COLORS[vedo_color]
                color = int(color_hex.lstrip('#'), 16)
            else:
                color = 0x00FF00
        elif isinstance(vedo_color, (list, tuple, np.ndarray)):
            # Handle RGB tuples/arrays (values 0-1 or 0-255)
            rgb = np.asarray(vedo_color[:3])
            # Normalize to 0-255 range if needed
            if rgb.max() <= 1.0:
                rgb = (rgb * 255).astype(int)
            else:
                rgb = rgb.astype(int)
            r, g, b = int(rgb[0]), int(rgb[1]), int(rgb[2])
            color = (r << 16) | (g << 8) | b
        else:
            color = 0x00FF00
    
    return k3d.line(points, color=color, width=width * line_scale_factor)


In [ ]:
## Plot optimization results with k3d
plot = k3d.plot(name=f'Optimized Path (w_obs={w_obstacle_norm:.2f}, w_len={w_path_length_norm:.2f})')

if merged_full_brain:
    plot += vedo_to_k3d_mesh(merged_full_brain)

if selected_hemisphere == 'left':
    print("Left hemisphere focused.")
    if merged_left_entry_zone:
        plot += vedo_to_k3d_mesh(merged_left_entry_zone, opacity_factor=2)
    if merged_right_entry_zone:
        plot += vedo_to_k3d_mesh(merged_right_entry_zone, opacity_factor=0.5)
    if merged_left_stn:
        plot += vedo_to_k3d_mesh(merged_left_stn, opacity_factor=1)
    if merged_right_stn:
        plot += vedo_to_k3d_mesh(merged_right_stn, opacity_factor=0.5)
elif selected_hemisphere == 'right':
    print("Right hemisphere focused.")
    if merged_left_entry_zone:
        plot += vedo_to_k3d_mesh(merged_left_entry_zone, opacity_factor=0.5)
    if merged_right_entry_zone:
        plot += vedo_to_k3d_mesh(merged_right_entry_zone, opacity_factor=2)
    if merged_left_stn:
        plot += vedo_to_k3d_mesh(merged_left_stn, opacity_factor=0.5)
    if merged_right_stn:
        plot += vedo_to_k3d_mesh(merged_right_stn, opacity_factor=1)
else:
    if merged_left_entry_zone:
        plot += vedo_to_k3d_mesh(merged_left_entry_zone)
    if merged_right_entry_zone:
        plot += vedo_to_k3d_mesh(merged_right_entry_zone)
    if merged_left_stn:
        plot += vedo_to_k3d_mesh(merged_left_stn)
    if merged_right_stn:
        plot += vedo_to_k3d_mesh(merged_right_stn)

if merged_obstacles:
    plot += vedo_to_k3d_mesh(merged_obstacles)

for line in failed_lines:
    plot += vedo_line_to_k3d(line, color=0xFF5F1F, width=FAILED_PATH_WIDTH * 0.5)
for line in suboptimal_lines:
    plot += vedo_line_to_k3d(line, color=0xFFFF00, width=SUBOPTIMAL_PATH_WIDTH * 0.5)
if best_path:
    plot += vedo_line_to_k3d(best_path, color=0x00FF00, width=BEST_PATH_WIDTH * 0.7)

try:
    plot += vedo_line_to_k3d(nm_optimal_line, color=0x00FFFF, width=BEST_PATH_WIDTH * 1.0)
except: 
    print("Nelder-Mead optimized path not found.")

print("Displaying optimization result with k3d...")
plot.display()
